Finetune ViT on CIFAR-10

# Necessary Libraries

In [1]:
%pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" an

# Model Checkpoint

In [1]:
model_checkpt = "google/vit-base-patch16-224-in21k" # May be better for fine tuning than non-in21k version

# Load Dataset

In [2]:
from datasets import load_dataset

cifar_10_ds = load_dataset("cifar10")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
import evaluate

metric = evaluate.load("accuracy")

In [4]:
label2id = {label: idx for idx, label in enumerate(cifar_10_ds["train"].features["label"].names)}
id2label = {idx: label for label, idx in label2id.items()}

# Pre-process Dataset

Note: Data augmentation is much more important for Vision Transformers than CNNs.

In [5]:
from transformers import AutoImageProcessor

image_processor = AutoImageProcessor.from_pretrained(model_checkpt) # Make sure image resize is apppropriate for model

image_processor

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


ViTImageProcessor {
  "do_convert_rgb": null,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [6]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std) # Normalize color channels

if "height" in image_processor.size: # Crop images to size model expects
    size = (image_processor.size["height"], image_processor.size["width"])
    crop_size = size
    max_size = None

elif "shortest_edge" in image_processor.size:
    size = image_processor.size["shortest_edge"]
    crop_size = (size, size)
    max_size = image_processor.size.get("longest_edge")


train_transforms = Compose(
    [
        RandomResizedCrop(crop_size),
        RandomHorizontalFlip(),
        ToTensor(),
        normalize,
    ]
)

validation_transforms = Compose(
    [
        Resize(size),
        CenterCrop(crop_size),
        ToTensor(),
        normalize,
    ]
)

In [7]:
def preprocess_train(example_batch):
    """
    Apply transformations to images in batch
    """
    example_batch["pixel_values"] = [train_transforms(image.convert("RGB")) for image in example_batch["img"]]
    return example_batch

def preprocess_validation(example_batch):
    """
    Apply transformations to images in batch
    """
    example_batch["pixel_values"] = [validation_transforms(image.convert("RGB")) for image in example_batch["img"]]
    return example_batch

In [8]:
# Apply transformations to dataset

splits = cifar_10_ds["train"].train_test_split(test_size=0.2)

train_ds = splits["train"]
val_ds = splits["test"]

train_ds.set_transform(preprocess_train) # Set_transform only applies when images loaded into RAM, so better to use than map
val_ds.set_transform(preprocess_validation)

In [9]:
train_ds[0] # Make sure pixel values feature has been added

{'img': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>,
 'label': 3,
 'pixel_values': tensor([[[-0.0588, -0.0588, -0.0588,  ..., -0.1843, -0.1843, -0.1843],
          [-0.0588, -0.0588, -0.0588,  ..., -0.1843, -0.1843, -0.1843],
          [-0.0588, -0.0588, -0.0588,  ..., -0.1843, -0.1843, -0.1843],
          ...,
          [-0.5922, -0.5922, -0.5922,  ..., -0.7412, -0.7412, -0.7412],
          [-0.5922, -0.5922, -0.5922,  ..., -0.7412, -0.7412, -0.7412],
          [-0.5922, -0.5922, -0.5922,  ..., -0.7412, -0.7412, -0.7412]],
 
         [[ 0.0353,  0.0353,  0.0353,  ..., -0.0588, -0.0588, -0.0588],
          [ 0.0353,  0.0353,  0.0353,  ..., -0.0588, -0.0588, -0.0588],
          [ 0.0353,  0.0353,  0.0353,  ..., -0.0588, -0.0588, -0.0588],
          ...,
          [-0.5059, -0.5059, -0.5059,  ..., -0.6549, -0.6549, -0.6549],
          [-0.5059, -0.5059, -0.5059,  ..., -0.6549, -0.6549, -0.6549],
          [-0.5059, -0.5059, -0.5059,  ..., -0.6549, -0.6549, -0.6549]],
 
  

# Fine Tuning

## Load Model

In [10]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained( # Load model
    model_checkpt,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True, # Need b/c fine-tuning from already fine-tuned model checkpoint
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Set Hyperparameters

In [11]:
from logging import log
model_name = model_checkpt.split("/")[-1] # Remove google/ part for finetuned model name

output_dir = f"{model_name}-finetuned-cifar10"
batch_size = 32 # Increase to speed up training & evaluation
logging_steps = 10 # Get more granular updates

training_args = TrainingArguments(
    output_dir=output_dir,
    remove_unused_columns=False, # Need b/c image column is necessary to create pixel values
    eval_strategy="epoch",
    save_strategy="epoch", # Save model at each epoch
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4, # Speeds up training
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    warmup_ratio=0.1, # 10% of training steps to increase learning rate to desired amount
    logging_steps=logging_steps,
    disable_tqdm=False,
    report_to="none", # Disable WandB logging
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True, # Speeds up training
)

## Define Accuracy and Collation Functions

In [12]:
import numpy as np

def compute_metrics(eval_pred):
    """
    Compute accuracy metrics on a batch of predictions
    """
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
import torch

def collate(examples: list):
  """
  Create batches
  """

  # Stack all pixel values into single tensor
  pixel_values = torch.stack([example["pixel_values"] for example in examples])

  # Extract labels from each example
  labels = torch.tensor([example["label"] for example in examples])

  return {"pixel_values": pixel_values, "labels": labels}

## Train and Evaluate Model

In [14]:
trainer = Trainer(
    model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    processing_class=image_processor, # Make sure image processor JSON config is saved w/ model
    compute_metrics=compute_metrics,
    data_collator=collate
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.416600,0.232361,0.979100
2,0.266100,0.112587,0.987700


TrainOutput(global_step=936, training_loss=0.5461161116249541, metrics={'train_runtime': 1354.7959, 'train_samples_per_second': 88.574, 'train_steps_per_second': 0.691, 'total_flos': 9.27490629380604e+18, 'train_loss': 0.5461161116249541, 'epoch': 2.992})

# Upload model to Hugging Face Hub

In [15]:
from huggingface_hub import notebook_login

notebook_login() # Login to account

In [18]:
kwargs = {
    "dataset_tags": "cifar-10",
    "dataset": "cifar-10",
}

trainer.push_to_hub(commit_message="Improve model accuracy", **kwargs)

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/avanishd/vit-base-patch16-224-in21k-finetuned-cifar10/commit/2b40a4381a86cbf3b404c6fd8e45da6d4d472446', commit_message='Improve model accuracy', commit_description='', oid='2b40a4381a86cbf3b404c6fd8e45da6d4d472446', pr_url=None, repo_url=RepoUrl('https://huggingface.co/avanishd/vit-base-patch16-224-in21k-finetuned-cifar10', endpoint='https://huggingface.co', repo_type='model', repo_id='avanishd/vit-base-patch16-224-in21k-finetuned-cifar10'), pr_revision=None, pr_num=None)